### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes five files: 

1. A file with one record for each airport, 
2. A file with one record for each airline, 
3. A file with one record for each each route, 
4. A file with one record for each type of airplane, and
5. A file with one record for each country.  

Each file has a relationship to the others, with some fields in common.  To make best use of the data, we will need to combine all three files.  This is the core challenge of relational databases.  This lesson looks at how to do that, specifically using the merge() function in python.  

Credits:

This lesson draws from the Data Analysis and Visualization in Python for Ecologists course by Data Carpentry:

https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html

and from the pandas documentation:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### Learning Objectives: 

1. Understand the meaning of relational data and be able to identify common keys. 

2. Understand the different types of relational joins: concatenate, inner join, outer join, left join, right join. 

3. Use the pandas syntax pd.concat() and pd.merge() to perform those relational joins. 

4. Start to build an awareness of open data resources, and of documentation/tutorials for ongoing learning. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["ID", "NAME", "CITY", "COUNTRY", "IATA", "ICAO", "LATITUDE", "LONGITUDE", "ALTITUDE", "TIMEZONE", "DST", "TZ", "TYPE", "SOURCE"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["ID", "NAME", "ALIAS", "IATA", "ICAO", "CALLSIGN", "COUNTRY", "ACTIVE"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["AIRLINE", "AIRLINE_ID", "SOURCE", "SOURCE_ID", "DEST", "DEST_ID", "CODESHARE", "STOPS", "EQUIPMENT"]

# Read in the planes data.
planes = pd.read_csv("data/planes.dat", header=None, na_values='\\N')
planes.columns = ["NAME", "IATA", "ICAO"]

# Read in the countries data.
countries = pd.read_csv("data/countries.dat", header=None, na_values='\\N')
countries.columns = ["NAME", "ISO", "DAFIF"]

### Start by seeing what's in the data.  

What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

Start with the airports table

In [3]:
# printing the first few rows gives us a good sense of what is in there
airports.head()

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
# we can also look a the data types
# alternatively, we could summarize the data as we did in the previous lesson
airports.dtypes

ID             int64
NAME          object
CITY          object
COUNTRY       object
IATA          object
ICAO          object
LATITUDE     float64
LONGITUDE    float64
ALTITUDE       int64
TIMEZONE     float64
DST           object
TZ            object
TYPE          object
SOURCE        object
dtype: object

##### Your turn:  

Spend a few minutes examining the remaining tables.  See if you can identify which fields are common across multiple tables.  These are the "keys".  Together, we will make sure we have identified the right keys. 

In [5]:
airlines.dtypes

ID           int64
NAME        object
ALIAS       object
IATA        object
ICAO        object
CALLSIGN    object
COUNTRY     object
ACTIVE      object
dtype: object

In [6]:
routes.dtypes

AIRLINE        object
AIRLINE_ID    float64
SOURCE         object
SOURCE_ID     float64
DEST           object
DEST_ID       float64
CODESHARE      object
STOPS           int64
EQUIPMENT      object
dtype: object

In [7]:
planes.dtypes

NAME    object
IATA    object
ICAO    object
dtype: object

In [8]:
countries.dtypes

NAME     object
ISO      object
DAFIF    object
dtype: object

### Selecting data

We might want to work with a subset of data.  If so, we can select data just like we did in the previous lesson. 

In [9]:
# we can select all airports in one of the least visited countries on earth
airports[airports['COUNTRY']=="Tuvalu"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3835,4077,Funafuti International Airport,Funafuti,Tuvalu,FUN,NGFU,-8.525,179.195999,9,12.0,U,Pacific/Funafuti,airport,OurAirports


In [10]:
# we can select all routes to that airport
routes[routes['DEST']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5


In [11]:
# and routes from that airport
routes[routes['SOURCE']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [ ]:
# where are these flights going to/from? 


#####  Your turn: 

Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [19]:
from_lex = routes[routes['SOURCE'] == 'LEX']
from_lex.shape[0]

20

In [20]:
to_lex = routes[routes['DEST'] == 'LEX']
to_lex.shape[0]

20

### Concatentating data

To concatenate is to link things together in a series or chain.  It is similar to, but more powerful than appending data, because it can work on either rows or columns, is faster, and will perform set logic (union or intersection).  

In [21]:
# let's go back and contatenate the routes both to and from Tuvalu
to_tuvalu = routes[routes['DEST']=='FUN']
from_tuvalu = routes[routes['SOURCE']=='FUN']


In [22]:
# the basic function, given a list of dataframes to concatenate
# note that we call concatenate as a "static" method, rather than a method 
# associated with a specific object. 

pd.concat([to_tuvalu, from_tuvalu])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [23]:
# if we want to use the dataframe later, we need to assign it to a variable name

tuvalu_routes = pd.concat([to_tuvalu, from_tuvalu])

Note to see more about this method, we can examine the pandas docs:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [24]:
# note that one of the arguments is axis.  
# axis=0 indicates that the rows should be concatenated, 
# while axis=1 indicates to concatenate the columns 

pd.concat([to_tuvalu, from_tuvalu], axis=1)

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Your turn:

Can you use the concat() function to create one dataframe with flights both into and out of LEX?

In [25]:
pd.concat([to_lex, from_lex])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


### Merging data

Sometimes we want to merge, or join, the data instead of concatenating it.  For example, maybe we hate flying on small planes, so want to know what kind of aircraft we would need to fly on to visit Tuvalu.  This data comes from a different table. 

In [26]:
# we see that the Tuvalu routes operate with AT5 equipment
tuvalu_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [27]:
# but what is an AT5? Let's look in the planes table. 
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [28]:
# in this case, we are lucky, and it is in the first few rows.  But to do this systemactically, we
# would want to join the tables.  To do this, we use the merge() function in pandas.  

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


Let's stop and look at what is going on in the above command. 

It is important to understand the type of merge, as specified by the how option.  Here is what the pandas docs say:


![alt text](img/how.png)


![alt text](img/how2.png)

So it looks like the above merge defaulted to an inner join


![alt text](img/inner.png)

In [29]:
# we can check this by explicitly setting the option and confirming that we get the same result  
# next, let's check what happens if we choose the other options

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='inner')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [30]:
# outer join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='outer')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-134,TU3,T134
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


In [31]:
# left join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='left')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [32]:
# right join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='right')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-144,NaN,T144
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


I find that I most often want a left join, as described here: 

![alt text](img/left.png)

It may be worth examining some of the other options used in merge().  We can refer to the pandas documentation to learn more. 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

#### Your turn

Now let's look at which airlines operate in and out of Lexington.  How many routes does each airline have?  

First, let's spend some time planning together.  Then individually take some time to implement the code we planned. 

In [35]:
from_lex = routes[routes['SOURCE'] == 'LEX']
to_lex = routes[routes['DEST'] == 'LEX']
tofrom_lex = pd.concat([to_lex, from_lex])
grouped = tofrom_lex['AIRLINE'].groupby(tofrom_lex['AIRLINE'])
grouped.count()

AIRLINE
9E     2
AA     6
AF     2
DL    10
G4     8
KL     2
UA     4
US     6
Name: AIRLINE, dtype: int64

#### It looks like there are some international airlines with Lexington routes.  

To look at how many routes they have, create a new column in your dataframe called 'International', which is set to True for an overseas airline and False for a domestic airline.  Calculate the percent of routes with an overseas airline.  

You should have the tools you need to answer this question using what we learned in last week's lesson.  Again, let's start by planning together, then implement the code on your own.  

In [36]:
from_lex = routes[routes['SOURCE'] == 'LEX']
to_lex = routes[routes['DEST'] == 'LEX']
tofrom_lex = pd.concat([to_lex, from_lex])

tofrom_lex =pd.merge(tofrom_lex, airlines, left_on='AIRLINE_ID', right_on='ID', how='left')
US = tofrom_lex['COUNTRY']=='United States'
tofrom_lex = pd.concat([tofrom_lex,US], axis=1)
tofrom_lex

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE,COUNTRY
0,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y,True
1,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,True
2,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,True
3,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,True
4,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,137,Air France,NaN,AF,AFR,AIRFRANS,France,Y,False
5,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,True
6,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,True
7,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,True
8,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,True
9,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,True


#### Codeshares

It looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

It is this sort of messiness that will consume much of your effort in research and data science.  It is very important to "become one with your data". 

Figuring this out will be the first part of your homework.  

### Review Questions: 

1. What is a key in relational data? 

2. Explain the difference between a left join and an outer join.  

3. Explain the difference between pd.concat() and pd.merge()

4. Identify two possible places to look for help if you get stuck on the homework. 

### Homework 1 - Code shares

Dig into the data as described above, and do your best to identify which routes are code shares, and how many unique routes operate into/out of Lexington.

### Homework 2 - The Kevin Bacon Game

Of course, it is possible to fly beyond those few cities from Lexington--you just have to transfer to get there.  Your task is to:

1. Identify which airports you can get to from Lexington with only one transfer.  
2. Identify which airports you can get to from Lexington with two transfers.

You have the tools to do this using the merge functionality presented today, but you will need to be smart about what you are merging.  

Next class, we will use these data to make interactive maps of these flight routes, similar to what you see in the back of the airline magazines.  


#### Bonus: 

This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/). What is the number N, such that you can reach every airport in the world with N or fewer transfers?

#### Extra Bonus: 

Use this very important piece of knowledge to impress your friends at parties!